# Plot the time of saccade vs. the length of a saccade

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from utils import (
    DEGREES_PER_PIXEL,
    TIMESTAMP_IDENT,
    X_PIXELS,
    Y_PIXELS,
    Events,
    extract_gaze_data_between_timestamps_proper,
    get_participant_dominant_eye,
)
import json

from velocityThreshold import detect_fix_ivt, find_sacc_from_fix

EYE_TRACKER_FOLDER = "eye_tracker_data/"
GAZE_DATA = []
GAZE_DATA_BOOK = []
GAZE_DATA_PAGE = []

# participants that were told that they can change the settings beforehand
participant_ids = [11]
subquery = Events.select().where(Events.participant_id.in_(participant_ids))
books = subquery.where(Events.event == "OPEN_BOOK")

for participant in participant_ids:
    for book in books:
        START_TIME_BOOK = book.time
        book_end = (
            subquery.where(Events.event == "CLOSE_BOOK")
            .where(Events.time > START_TIME_BOOK)
            .get()
        )
        END_TIME_BOOK = book_end.time

        # Get the events that have a timestamp less than the first CLOSE_BOOK event
        events_book = subquery.where(Events.time <= END_TIME_BOOK)

        formatted_time = datetime.fromtimestamp(START_TIME_BOOK / 1000).strftime(
            "%Y-%m-%d_%H-%M-%S"
        )
        GAZE_FILE = f"{EYE_TRACKER_FOLDER}[{participant}]-{formatted_time}.json"
        f = open(GAZE_FILE, "r")
        GAZE_DATA_BOOK = json.load(f)
        f.close()

        pages = (
            subquery.where(Events.event == "NEXT_PAGE")
            .where(Events.time > START_TIME_BOOK)
            .where(Events.time < END_TIME_BOOK)
        )
        for page in pages:
            START_TIME_PAGE = page.time
            end_page = subquery.where(Events.event == "NEXT_PAGE").where(
                Events.time > START_TIME_PAGE
            )
            if end_page.count() == 0:
                continue
            END_TIME_PAGE = (end_page.get()).time

            # Get the gaze data for the page
            GAZE_DATA_PAGE = extract_gaze_data_between_timestamps_proper(
                GAZE_DATA_BOOK, START_TIME_PAGE, END_TIME_PAGE
            )

            # print(GAZE_DATA_PAGE)

            timestamps = []
            x = []
            y = []

            DOMINANT_EYE = get_participant_dominant_eye(participant)

            # for each packet, plot the gaze point
            for packet in GAZE_DATA_PAGE["data"]:
                if packet[f"{DOMINANT_EYE}_gaze_point_validity"] == 0:
                    continue
                x.append(
                    (packet[f"{DOMINANT_EYE}_gaze_point_on_display_area"][0] * X_PIXELS)
                    # * DEGREES_PER_PIXEL
                )
                y.append(
                    (packet[f"{DOMINANT_EYE}_gaze_point_on_display_area"][1] * Y_PIXELS)
                    # * DEGREES_PER_PIXEL
                )
                timestamps.append(packet[TIMESTAMP_IDENT])

            df = pd.DataFrame({"x": x, "y": y, "ts": timestamps})
            df = df.sort_values(by="ts")
            df = df.reset_index(drop=True)

            df["x"] = df["x"] * DEGREES_PER_PIXEL
            df["y"] = df["y"] * DEGREES_PER_PIXEL
            df["ts"] = df["ts"] / 1_000_000

            # display(df)
            SACCADIC_THRESHOLD = 80
            # Plot fixations
            fixations, v, labels = detect_fix_ivt(df, sacvel=SACCADIC_THRESHOLD)
            saccades = find_sacc_from_fix(fixations)
            # print(saccades)

            fig, ax = plt.subplots(figsize=(X_PIXELS / 100, Y_PIXELS / 100))
            # Make x axis go from 0 to 50
            # ax.set_xlim(0, 50)
            ax.set_ylim(0, 0.1)

            ax.set_title('Main Sequence')
            p = ax.scatter(
                saccades["dxy"],
                saccades["len"],
                c=saccades["ts"],
                s=40,
                cmap="plasma",
                label=f"Participant {participant} - {book.new_value}",
            )


            # Make trendline
            z = np.polyfit(saccades["dxy"], saccades["len"], 1)
            p = np.poly1d(z)
            # Label with equation
            plt.plot(saccades["dxy"], p(saccades["dxy"]), "r--", label="f(x) = %.6fx + %.6f"%(z[0],z[1]))

            ax.set_xlabel("Distance (degrees)")
            ax.set_ylabel("Duration (seconds)")
            ax.legend()

In [40]:
# Get the average for each page fitted linear model of the main sequences of a participant
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from utils import (
    DEGREES_PER_PIXEL,
    TIMESTAMP_IDENT,
    X_PIXELS,
    Y_PIXELS,
    Events,
    extract_gaze_data_between_timestamps_proper,
    get_participant_dominant_eye,
)
import json


from velocityThreshold import detect_fix_ivt, find_sacc_from_fix

EYE_TRACKER_FOLDER = "eye_tracker_data/"
GAZE_DATA = []
GAZE_DATA_BOOK = []
GAZE_DATA_PAGE = []

# participants that were told that they can change the settings beforehand
participant_ids = [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
subquery = Events.select().where(Events.participant_id.in_(participant_ids))
books = subquery.where(
    (Events.event == "OPEN_BOOK") & (Events.new_value != "Smart Farming Tech - ")
)

for book in books:
    START_TIME_BOOK = book.time
    book_end = (
        subquery.where(Events.event == "CLOSE_BOOK")
        .where(Events.time > START_TIME_BOOK)
        .get()
    )
    END_TIME_BOOK = book_end.time

    # Get the events that have a timestamp less than the first CLOSE_BOOK event
    events_book = subquery.where(Events.time <= END_TIME_BOOK)

    # Read from participants.json whether or not the participant has low resolution data
    # These we should not divide the timestamp with 1000

    LOW_RES = json.load(open("participants.json", "r"))[f"{book.participant_id}"][
        "low_resolution"
    ]
    if LOW_RES:  # TODO: use the estimate from the stdev
        continue

    if not LOW_RES:
        START_TIME_BOOK /= 1000
    formatted_time = datetime.fromtimestamp(START_TIME_BOOK).strftime(
        "%Y-%m-%d_%H-%M-%S"
    )
    GAZE_FILE = f"{EYE_TRACKER_FOLDER}[{book.participant_id}]-{formatted_time}.json"
    f = open(GAZE_FILE, "r")
    GAZE_DATA_BOOK = json.load(f)
    f.close()

    DOMINANT_EYE = get_participant_dominant_eye(book.participant_id)

    pages = subquery.where(Events.event == "NEXT_PAGE").where(
        (Events.time > START_TIME_BOOK) & (Events.time < END_TIME_BOOK)
    )
    for page in pages:
        START_TIME_PAGE = page.time
        end_page = subquery.where(Events.event == "NEXT_PAGE").where(
            Events.time > START_TIME_PAGE
        )
        if end_page.count() == 0:
            continue
        END_TIME_PAGE = (end_page.get()).time

        # Get the gaze data for the page
        GAZE_DATA_PAGE = extract_gaze_data_between_timestamps_proper(
            GAZE_DATA_BOOK, START_TIME_PAGE, END_TIME_PAGE
        )

        # print(GAZE_DATA_PAGE)

        timestamps = []
        x = []
        y = []

        # for each packet, plot the gaze point
        for packet in GAZE_DATA_PAGE["data"]:
            if packet[f"{DOMINANT_EYE}_gaze_point_validity"] == 0:
                continue
            x.append(
                (packet[f"{DOMINANT_EYE}_gaze_point_on_display_area"][0] * X_PIXELS)
                # * DEGREES_PER_PIXEL
            )
            y.append(
                (packet[f"{DOMINANT_EYE}_gaze_point_on_display_area"][1] * Y_PIXELS)
                # * DEGREES_PER_PIXEL
            )
            timestamps.append(packet[TIMESTAMP_IDENT])

        df = pd.DataFrame({"x": x, "y": y, "ts": timestamps})
        df = df.sort_values(by="ts")
        df = df.reset_index(drop=True)

        df["x"] = df["x"] * DEGREES_PER_PIXEL
        df["y"] = df["y"] * DEGREES_PER_PIXEL
        df["ts"] = df["ts"] / 1_000_000

        # display(df)
        SACCADIC_THRESHOLD = 80
        # Plot fixations
        fixations, v, labels = detect_fix_ivt(df, sacvel=SACCADIC_THRESHOLD)
        saccades = find_sacc_from_fix(fixations)

        z = np.polyfit(saccades["dxy"], saccades["len"], 1)
        p = np.poly1d(z)

        # Continue from here by saving the 1 dimensional (linear) polyfit into a map or something
        # and then averaging them on a book and participant basis

/Users/idk/repos/flow/apps/backend/velocityThreshold.py:62: RuntimeWarning: invalid value encountered in scalar divide
  rho = cov[0,1] / (sx*sy)
/Users/idk/repos/flow/apps/backend/velocityThreshold.py:62: RuntimeWarning: invalid value encountered in scalar divide
  rho = cov[0,1] / (sx*sy)
/Users/idk/repos/flow/apps/backend/velocityThreshold.py:62: RuntimeWarning: invalid value encountered in scalar divide
  rho = cov[0,1] / (sx*sy)
/Users/idk/repos/flow/apps/backend/velocityThreshold.py:62: RuntimeWarning: invalid value encountered in scalar divide
  rho = cov[0,1] / (sx*sy)
/Users/idk/repos/flow/apps/backend/velocityThreshold.py:62: RuntimeWarning: invalid value encountered in scalar divide
  rho = cov[0,1] / (sx*sy)
/Users/idk/repos/flow/apps/backend/velocityThreshold.py:62: RuntimeWarning: invalid value encountered in scalar divide
  rho = cov[0,1] / (sx*sy)
/Users/idk/repos/flow/apps/backend/velocityThreshold.py:62: RuntimeWarning: invalid value encountered in scalar divide
  rho